In [120]:
import pandas as pd, numpy as np

# Compile regression estimates / examine for logical consistency

In [121]:
w_metadata_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data_with_metadata.csv'


In [122]:
w_metadata_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data_with_metadata.csv'
df = pd.read_csv(w_metadata_path)

In [123]:
df.location_id.unique()

array([179, 163, 214, 161, 165, 190, 189, 196, 201, 180,  15, 164,  20,
       202, 205, 171,  11, 184, 168,   6, 141, 207, 181, 213, 522])

In [124]:
df.estimation_status.unique()

array(['regression', 'na', 'multiplicative', 'lack of evidence'],
      dtype=object)

In [125]:
df[(df.value_mean.isna())]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable,estimation_status,source_citation,source_link,CI_source,data_choice_notes


In [126]:
df[df.value_025_percentile.isna()].location_name.unique()

array(['Angola', 'China', 'Egypt', 'Ghana', 'Madagascar', 'Niger',
       'Sudan'], dtype=object)

In [127]:
A = df[(df.value_description=="percent of population eating vehicle")].rename(columns={'value_mean':"A",'estimation_status':'A_estimation_status'})
B = df[(df.value_description=="percent of population eating industrially produced vehicle")].rename(columns={'value_mean':"B",'estimation_status':'B_estimation_status'})
C = df[(df.value_description=="percent of population eating fortified vehicle")].rename(columns={'value_mean':"C",'estimation_status':'C_estimation_status'})

A = A.drop(columns=['value_description','nutrient'])
B = B.drop(columns=['value_description','nutrient'])
C = C.drop(columns=['value_description'])

ref = A.merge(B, on = ['location_id','location_name','vehicle'], how = 'left').merge(C, on = ['location_id','location_name','vehicle'], how = 'outer')[['location_name','vehicle','nutrient','A','A_estimation_status','B','B_estimation_status','C','C_estimation_status']]


In [128]:
ref

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status
0,Ethiopia,maize flour,folic acid,59.259825,regression,27.169813,regression,0.000000,regression
1,Ethiopia,maize flour,iron,59.259825,regression,27.169813,regression,0.000000,regression
2,Ethiopia,maize flour,vitamin a,59.259825,regression,27.169813,regression,0.000000,regression
3,Ethiopia,maize flour,zinc,59.259825,regression,27.169813,regression,0.000000,regression
4,Ethiopia,oil,vitamin a,29.350000,regression,70.000000,na,0.000000,regression
...,...,...,...,...,...,...,...,...,...
211,Sudan,oil,vitamin a,50.646725,regression,80.000000,na,80.000000,na
212,Sudan,wheat flour,folic acid,88.758036,regression,35.365713,regression,1.760469,regression
213,Sudan,wheat flour,iron,88.758036,regression,35.365713,regression,1.760469,regression
214,Sudan,wheat flour,vitamin a,88.758036,regression,35.365713,regression,1.760469,regression


In [129]:
ref[(ref.B < ref.C)]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status
73,Burkina Faso,oil,vitamin a,92.230000,na,51.975594,multiplicative,75.000000,na
87,Kenya,wheat flour,folic acid,36.580000,na,30.950000,na,34.703333,multiplicative
88,Kenya,wheat flour,iron,36.580000,na,30.950000,na,33.835750,multiplicative
89,Kenya,wheat flour,vitamin a,36.580000,na,30.950000,na,34.703333,multiplicative
90,Kenya,wheat flour,zinc,36.580000,na,30.950000,na,34.703333,multiplicative
124,Côte d'Ivoire,maize flour,folic acid,84.907643,regression,5.387876,regression,22.793768,regression
125,Côte d'Ivoire,maize flour,iron,84.907643,regression,5.387876,regression,22.793768,regression
126,Côte d'Ivoire,maize flour,vitamin a,84.907643,regression,5.387876,regression,22.793768,regression
127,Côte d'Ivoire,maize flour,zinc,84.907643,regression,5.387876,regression,22.793768,regression
129,Côte d'Ivoire,wheat flour,folic acid,54.700000,na,10.200000,na,10.599703,regression


In [130]:
ref[(ref.B.isna()) | (ref.C.isna())]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status


In [131]:
ref.location_name.nunique()

25

In [132]:
oil_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_oil_regression_estimates_3_23_2021.csv'
wheat_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_wheat_regression_estimates_3_22_2021.csv'
maize_est_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_maize_regression_estimates_3_22_2021.csv'

ip_wheat_path =  '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_wheat_regression_estimates_3_31_2021.csv'
ip_oil_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_oil_regression_estimates_3_31_2021.csv'
ip_maize_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortifiable_maize_regression_estimates_3_31_2021.csv'

fort_wheat_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_wheat_regression_estimates_3_31_2021.csv'
fort_maize_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_maize_regression_estimates_3_31_2021.csv'
fort_oil_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/pct_eating_fortified_oil_regression_estimates_3_31_2021.csv'

In [133]:
paths_A = [oil_est_path,wheat_est_path,maize_est_path]
paths_B = [ip_wheat_path,ip_oil_path,ip_maize_path]
paths_C = [fort_wheat_path, fort_maize_path, fort_oil_path]

def format_data(paths, var, n):
    draws = [f'draw_{i}' for i in range(n)]
    df = pd.concat([pd.read_csv(path) for path in paths])
    df[var] = df[draws].mean(axis=1)
    df = df.drop(columns=draws)
    
    return df.groupby(['location_name','vehicle']).mean().reset_index()

In [134]:
scale_over_mean_map = {
    "maize flour":0.578996, 
    "oil":0.069702,
    "wheat flour":0.507994
}

In [135]:
df['scale_over_mean'] = 2 * df.vehicle.map(scale_over_mean_map)

In [136]:
eats = format_data(paths_A, 'reg_A', n = 1_000)
ip = format_data(paths_B, 'reg_B', n = 500)
fort = format_data(paths_C, 'reg_C',n = 500)

In [137]:
df = df.merge(eats, on = ['location_name','vehicle'], how = 'outer')

In [138]:
df['updated'] = False

In [139]:
df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating vehicle") & 
   (df.reg_A.notna()),'updated'] = True

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating vehicle") & 
   (df.reg_A.notna()),'estimation_status'] = 'regression'

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating vehicle") & 
   (df.reg_A.notna()),'value_mean'] = df.reg_A

In [140]:
df = df.merge(ip, on = ['location_name','vehicle'], how = 'outer')

In [141]:
df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating industrially produced vehicle") & 
   (df.reg_B.notna()),'updated'] = True

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating industrially produced vehicle") & 
   (df.reg_B.notna()),'estimation_status'] = 'regression'

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating industrially produced vehicle") & 
   (df.reg_B.notna()),'value_mean'] = df.reg_B

In [142]:
df = df.merge(fort, on = ['location_name','vehicle'], how = 'outer')

In [143]:
df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating fortified vehicle") & 
   (df.reg_C.notna()),'updated'] = True

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating fortified vehicle") & 
   (df.reg_B.notna()),'estimation_status'] = 'regression'

df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
   (df.value_description=="percent of population eating fortified vehicle") & 
   (df.reg_B.notna()),'value_mean'] = df.reg_C

In [144]:
df[(df.updated)].estimation_status.unique()

array(['regression'], dtype=object)

In [145]:
df['lower'] = np.clip(df.value_mean - (df.scale_over_mean * df.value_mean)/2, 0, 100)
df['upper'] = np.clip(df.value_mean + (df.scale_over_mean * df.value_mean)/2, 0, 100)

In [146]:
df.loc[(df.updated),'value_025_percentile'] = df.lower
df.loc[(df.updated),'value_975_percentile'] = df.upper

In [147]:
df.loc[(df.value_025_percentile.isna()),'value_025_percentile'] = df.lower
df.loc[(df.value_975_percentile.isna()),'value_975_percentile'] = df.upper

In [148]:
A = df[(df.value_description=="percent of population eating vehicle")].rename(
    columns={
        'value_mean':"A",
        'estimation_status':'A_estimation_status',
        'value_025_percentile':'A_lower',
        'value_975_percentile':'A_upper'
    })
B = df[(df.value_description=="percent of population eating industrially produced vehicle")].rename(
    columns={
        'value_mean':"B",
        'estimation_status':'B_estimation_status',
        'value_025_percentile':'B_lower',
        'value_975_percentile':'B_upper'
    })

C = df[(df.value_description=="percent of population eating fortified vehicle")].rename(
    columns={
        'value_mean':"C",
        'estimation_status':'C_estimation_status',
        'value_025_percentile':'C_lower',
        'value_975_percentile':'C_upper'
    })

In [149]:
# B = df[(df.value_description=="percent of population eating industrially produced vehicle")].rename(columns={'value_mean':"B",'estimation_status':'B_estimation_status'})
# C = df[(df.value_description=="percent of population eating fortified vehicle")].rename(columns={'value_mean':"C",'estimation_status':'C_estimation_status'})

A = A.drop(columns=['value_description','nutrient'])
B = B.drop(columns=['value_description','nutrient'])
C = C.drop(columns=['value_description'])

ref = A.merge(B, on = ['location_id','location_name','vehicle'], how = 'left').merge(C, on = ['location_id','location_name','vehicle'], how = 'outer') #[['location_name','vehicle','nutrient','A','A_estimation_status','B','B_estimation_status','C','C_estimation_status']]

In [150]:
ref = ref[['location_name','vehicle','nutrient',
           'A','A_lower','A_upper','A_estimation_status',
           'B','B_lower','B_upper','B_estimation_status',
           'C','C_lower','C_upper','C_estimation_status']]

In [151]:
ref

,location_name,vehicle,nutrient,A,A_lower,A_upper,A_estimation_status,B,B_lower,B_upper,B_estimation_status,C,C_lower,C_upper,C_estimation_status
0,Ethiopia,maize flour,folic acid,59.263848,24.950317,93.577380,regression,27.167514,11.437632,42.897396,regression,0.000000,0.000000,0.000000,regression
1,Ethiopia,maize flour,iron,59.263848,24.950317,93.577380,regression,27.167514,11.437632,42.897396,regression,0.000000,0.000000,0.000000,regression
2,Ethiopia,maize flour,vitamin a,59.263848,24.950317,93.577380,regression,27.167514,11.437632,42.897396,regression,0.000000,0.000000,0.000000,regression
3,Ethiopia,maize flour,zinc,59.263848,24.950317,93.577380,regression,27.167514,11.437632,42.897396,regression,0.000000,0.000000,0.000000,regression
4,Ethiopia,oil,vitamin a,29.350000,27.304246,31.395754,regression,70.000000,64.070691,75.929309,na,0.000000,0.000000,0.000000,regression
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Sudan,oil,vitamin a,50.646725,47.116547,54.176903,regression,80.000000,74.423840,85.576160,na,80.000000,74.423840,85.576160,na
212,Sudan,wheat flour,folic acid,88.823684,43.701786,100.000000,regression,57.902255,28.488257,87.316254,regression,6.525113,3.210395,9.839831,regression
213,Sudan,wheat flour,iron,88.823684,43.701786,100.000000,regression,57.902255,28.488257,87.316254,regression,6.525113,3.210395,9.839831,regression
214,Sudan,wheat flour,vitamin a,88.823684,43.701786,100.000000,regression,57.902255,28.488257,87.316254,regression,6.525113,3.210395,9.839831,regression


In [152]:
ref[(ref.B < ref.C)][['A','B','C','B_upper','C_lower']]

,A,B,C,B_upper,C_lower
73,92.230000,51.975594,75.000000,77.976411,37.481212
87,36.580000,30.950000,34.703333,48.296257,15.253480
88,36.580000,30.950000,33.835750,48.296257,14.872143
89,36.580000,30.950000,34.703333,48.296257,15.253480
90,36.580000,30.950000,34.703333,48.296257,15.253480
130,54.700000,10.200000,13.401500,13.100000,9.082841
143,75.761649,29.655911,73.100000,44.720935,49.543388
144,75.761649,29.655911,73.100000,44.720935,49.543388
146,75.761649,29.655911,73.100000,44.720935,49.543388
156,51.100278,9.158853,9.191133,14.461792,3.869504


In [153]:
ref[(ref.B.isna()) | (ref.C.isna())][['location_name','vehicle','nutrient','A_estimation_status','A','B_estimation_status','B','C_estimation_status','C']]

,location_name,vehicle,nutrient,A_estimation_status,A,B_estimation_status,B,C_estimation_status,C
145,Indonesia,wheat flour,vitamin a,regression,75.761649,regression,29.655911,regression,NaN
165,China,oil,vitamin a,regression,57.651373,regression,63.577005,regression,NaN
174,Egypt,oil,vitamin a,regression,54.973297,regression,70.101964,regression,NaN


In [154]:
ref[(ref.B_upper < ref.C_lower)][['location_name','vehicle','nutrient','A_estimation_status','A','B_estimation_status','B','C_estimation_status','C']]

,location_name,vehicle,nutrient,A_estimation_status,A,B_estimation_status,B,C_estimation_status,C
143,Indonesia,wheat flour,folic acid,regression,75.761649,regression,29.655911,na,73.1
144,Indonesia,wheat flour,iron,regression,75.761649,regression,29.655911,na,73.1
146,Indonesia,wheat flour,zinc,regression,75.761649,regression,29.655911,na,73.1


In [155]:
# these are the locations rejection sampling should be able to fix
sub = ref.loc[(ref.B < ref.C) & ~(ref.B_upper < ref.C_lower)]

In [156]:
sub

,location_name,vehicle,nutrient,A,A_lower,A_upper,A_estimation_status,B,B_lower,B_upper,B_estimation_status,C,C_lower,C_upper,C_estimation_status
73,Burkina Faso,oil,vitamin a,92.230000,46.091896,100.000000,na,51.975594,25.974777,77.976411,multiplicative,75.000000,37.481212,100.000000,na
87,Kenya,wheat flour,folic acid,36.580000,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative
88,Kenya,wheat flour,iron,36.580000,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,33.835750,14.872143,52.799357,multiplicative
89,Kenya,wheat flour,vitamin a,36.580000,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative
90,Kenya,wheat flour,zinc,36.580000,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative
130,Côte d'Ivoire,wheat flour,iron,54.700000,50.100000,59.600000,na,10.200000,7.500000,13.100000,na,13.401500,9.082841,17.720159,multiplicative
156,Angola,maize flour,folic acid,51.100278,21.513422,80.687135,regression,9.158853,3.855914,14.461792,regression,9.191133,3.869504,14.512762,regression
157,Angola,maize flour,iron,51.100278,21.513422,80.687135,regression,9.158853,3.855914,14.461792,regression,9.191133,3.869504,14.512762,regression
158,Angola,maize flour,vitamin a,51.100278,21.513422,80.687135,regression,9.158853,3.855914,14.461792,regression,9.191133,3.869504,14.512762,regression
159,Angola,maize flour,zinc,51.100278,21.513422,80.687135,regression,9.158853,3.855914,14.461792,regression,9.191133,3.869504,14.512762,regression


In [157]:
fortifiable = sub[['location_name','vehicle','nutrient','B','B_lower','B_upper','B_estimation_status']]
fortifiable['value_description'] = 'B'

fortified = sub[['location_name','vehicle','nutrient','C','C_lower','C_upper','C_estimation_status']]
fortified['value_description'] = 'C'

/share/code/beatrixh/miniconda/envs/stats/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/share/code/beatrixh/miniconda/envs/stats/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [158]:
import scipy.stats
import scipy.integrate as integrate

In [159]:
sub.head()

,location_name,vehicle,nutrient,A,A_lower,A_upper,A_estimation_status,B,B_lower,B_upper,B_estimation_status,C,C_lower,C_upper,C_estimation_status
73,Burkina Faso,oil,vitamin a,92.23,46.091896,100.000000,na,51.975594,25.974777,77.976411,multiplicative,75.000000,37.481212,100.000000,na
87,Kenya,wheat flour,folic acid,36.58,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative
88,Kenya,wheat flour,iron,36.58,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,33.835750,14.872143,52.799357,multiplicative
89,Kenya,wheat flour,vitamin a,36.58,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative
90,Kenya,wheat flour,zinc,36.58,16.078349,57.081651,na,30.950000,13.603743,48.296257,na,34.703333,15.253480,54.153186,multiplicative


In [160]:
replacement = pd.DataFrame()
for i, row in ref.loc[(ref.B < ref.C) & ~(ref.B_upper < ref.C_lower)].iterrows():
    std_b = np.max([(row.B_upper - row.B),(row.B - row.B_lower)]) / 1.96
    b_a = (0 - row.B) / std_b
    b_b = (100 - row.B) / std_b
    
    b_draws = scipy.stats.truncnorm.rvs(b_a, b_b, row.B, std_b, size = 500_000).tolist()
    
    std_c = np.max([(row.C_upper - row.C),(row.C - row.C_lower)]) / 1.96
    c_a = (0 - row.C) / std_c
    c_b = (100 - row.C) / std_c
    
    c_draws = scipy.stats.truncnorm.rvs(c_a, c_b, row.C, std_c, size = 500_000).tolist()
    
    all_draws = pd.DataFrame({
    'b':b_draws,
    'c':c_draws
    })

    all_draws = all_draws[all_draws.b > all_draws.c].T #only keep logical draws
    means = all_draws.mean(axis=1)
    lowers = all_draws.quantile(.025, axis=1)
    uppers = all_draws.quantile(.975, axis=1)
    
    
    new = pd.DataFrame({
    'location_name': [row.location_name] * 2,
    'vehicle': [row.vehicle] * 2,
    'nutrient': ['na',row.nutrient],
    'value_description':['percent of population eating industrially produced vehicle','percent of population eating fortified vehicle'],
    'value_mean':means,
    'value_025_percentile':lowers,
    'value_975_percentile':uppers,
    'estimation_status': [" + ".join([row.B_estimation_status,"rejection sampling"])," + ".join([row.C_estimation_status,"rejection sampling"])]
    })
    
    replacement = replacement.append(new)

In [161]:
replacement = replacement.groupby(['location_name','vehicle','value_description','nutrient','estimation_status']).mean().reset_index()

In [162]:
rename_repl = {
    'value_mean': 'new_value_mean',
    'value_025_percentile': 'new_value_025_percentile',
    'value_975_percentile': 'new_value_975_percentile',
    'estimation_status':'new_estimation_status'
}

replacement = replacement.rename(columns=rename_repl)

In [163]:
replacement

,location_name,vehicle,value_description,nutrient,new_estimation_status,new_value_mean,new_value_025_percentile,new_value_975_percentile
0,Angola,maize flour,percent of population eating fortified vehicle,folic acid,regression + rejection sampling,7.648049,3.125967,11.894459
1,Angola,maize flour,percent of population eating fortified vehicle,iron,regression + rejection sampling,7.648546,3.142450,11.902571
2,Angola,maize flour,percent of population eating fortified vehicle,vitamin a,regression + rejection sampling,7.655002,3.140304,11.888915
3,Angola,maize flour,percent of population eating fortified vehicle,zinc,regression + rejection sampling,7.654366,3.129934,11.893183
4,Angola,maize flour,percent of population eating industrially prod...,na,regression + rejection sampling,10.693961,6.457582,15.212412
5,Burkina Faso,oil,percent of population eating fortified vehicle,vitamin a,na + rejection sampling,51.148581,24.608298,75.341390
6,Burkina Faso,oil,percent of population eating industrially prod...,na,multiplicative + rejection sampling,63.392504,41.215125,85.806808
7,Côte d'Ivoire,wheat flour,percent of population eating fortified vehicle,iron,multiplicative + rejection sampling,10.310492,7.329621,13.015666
8,Côte d'Ivoire,wheat flour,percent of population eating industrially prod...,na,na + rejection sampling,11.600095,9.120707,14.101275
9,Kenya,wheat flour,percent of population eating fortified vehicle,folic acid,multiplicative + rejection sampling,27.455666,11.733148,42.143750


In [164]:
replacement['new_value_025_percentile'] = np.clip(replacement.new_value_025_percentile - 0.75*(replacement.new_value_mean - replacement.new_value_025_percentile), 0, 100)
replacement['new_value_975_percentile'] = np.clip(replacement.new_value_975_percentile + 0.75*(replacement.new_value_975_percentile - replacement.new_value_mean), 0, 100)

In [165]:
output = df.merge(replacement, on = ['location_name','vehicle','value_description','nutrient'], how = 'left')

In [166]:
output.loc[(output.new_value_mean.notna()),'value_mean'] = output.new_value_mean
output.loc[(output.new_value_mean.notna()),'value_025_percentile'] = output.new_value_025_percentile
output.loc[(output.new_value_mean.notna()),'value_975_percentile'] = output.new_value_975_percentile

In [167]:
output.loc[(output.new_value_mean.notna()),'estimation_status'] = output.new_estimation_status

In [168]:
final_cols = ['location_id', 'location_name', 'sub_population', 'vehicle',
       'value_description', 'nutrient', 'value_mean', 'value_025_percentile',
       'value_975_percentile']

In [169]:
all_inputted_data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data.csv'

In [170]:
output[output.value_mean.isna()]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,...,reg_A,updated,reg_B,reg_C,lower,upper,new_estimation_status,new_value_mean,new_value_025_percentile,new_value_975_percentile
218,11,Indonesia,NaN,wheat flour,percent of population eating fortified vehicle,vitamin a,NaN,25.986776,100.0,True,...,75.761649,False,29.655911,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,6,China,NaN,oil,percent of population eating fortified vehicle,vitamin a,NaN,NaN,NaN,True,...,57.651373,False,63.577005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,141,Egypt,NaN,oil,percent of population eating fortified vehicle,vitamin a,NaN,NaN,NaN,True,...,54.973297,False,70.101964,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
output[final_cols + ['u5_applicable','wra_applicable']]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,wra_applicable
0,179,Ethiopia,NaN,maize flour,percent of population eating vehicle,na,59.263848,24.950317,93.577380,True,True
1,179,Ethiopia,NaN,maize flour,percent of population eating industrially prod...,na,27.167514,11.437632,42.897396,True,True
2,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,folic acid,0.000000,0.000000,0.000000,True,True
3,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,iron,0.000000,0.000000,0.000000,True,True
4,179,Ethiopia,NaN,maize flour,percent of population eating fortified vehicle,vitamin a,0.000000,0.000000,0.000000,True,True
...,...,...,...,...,...,...,...,...,...,...,...
330,522,Sudan,NaN,wheat flour,percent of population eating industrially prod...,na,57.902255,28.488257,87.316254,True,True
331,522,Sudan,NaN,wheat flour,percent of population eating fortified vehicle,folic acid,6.525113,3.210395,9.839831,True,True
332,522,Sudan,NaN,wheat flour,percent of population eating fortified vehicle,iron,6.525113,3.210395,9.839831,True,True
333,522,Sudan,NaN,wheat flour,percent of population eating fortified vehicle,vitamin a,6.525113,3.210395,9.839831,True,True


In [172]:
# output[final_cols + ['u5_applicable','wra_applicable']].to_csv(all_inputted_data_path)

# output[final_cols + ['u5_applicable','wra_applicable']].to_csv('/ihme/homes/beatrixh/repos/scratch/lsff_input_coverage_data.csv')

In [173]:
A = output[(output.value_description=="percent of population eating vehicle")].rename(columns={'value_mean':"A",'estimation_status':'A_estimation_status'})
B = output[(output.value_description=="percent of population eating industrially produced vehicle")].rename(columns={'value_mean':"B",'estimation_status':'B_estimation_status'})
C = output[(output.value_description=="percent of population eating fortified vehicle")].rename(columns={'value_mean':"C",'estimation_status':'C_estimation_status'})

A = A.drop(columns=['value_description','nutrient'])
B = B.drop(columns=['value_description','nutrient'])
C = C.drop(columns=['value_description'])

ref2 = A.merge(B, on = ['location_id','location_name','vehicle'], how = 'left').merge(C, on = ['location_id','location_name','vehicle'], how = 'outer')[['location_name','vehicle','nutrient','A','A_estimation_status','B','B_estimation_status','C','C_estimation_status']]


In [174]:
ref2[(ref2.B < ref2.C)]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status
143,Indonesia,wheat flour,folic acid,75.761649,regression,29.655911,regression,73.1,na
144,Indonesia,wheat flour,iron,75.761649,regression,29.655911,regression,73.1,na
146,Indonesia,wheat flour,zinc,75.761649,regression,29.655911,regression,73.1,na


In [175]:
ref2[(ref2.B.isna() | ref2.C.isna())]

,location_name,vehicle,nutrient,A,A_estimation_status,B,B_estimation_status,C,C_estimation_status
145,Indonesia,wheat flour,vitamin a,75.761649,regression,29.655911,regression,NaN,regression
165,China,oil,vitamin a,57.651373,regression,63.577005,regression,NaN,regression
174,Egypt,oil,vitamin a,54.973297,regression,70.101964,regression,NaN,regression


In [176]:
## bug fixes

In [179]:
output[(output.value_mean.isna()) | (output.value_025_percentile.isna()) | (output.value_975_percentile.isna())]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,...,reg_A,updated,reg_B,reg_C,lower,upper,new_estimation_status,new_value_mean,new_value_025_percentile,new_value_975_percentile
218,11,Indonesia,NaN,wheat flour,percent of population eating fortified vehicle,vitamin a,NaN,25.986776,100.0,True,...,75.761649,False,29.655911,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,6,China,NaN,oil,percent of population eating fortified vehicle,vitamin a,NaN,NaN,NaN,True,...,57.651373,False,63.577005,NaN,NaN,NaN,NaN,NaN,NaN,NaN
267,141,Egypt,NaN,oil,percent of population eating fortified vehicle,vitamin a,NaN,NaN,NaN,True,...,54.973297,False,70.101964,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [180]:
all_inputted_data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data.csv'

In [181]:
output[(output.location_name=="Kenya") & (output.vehicle.isin(['maize flour','wheat flour'])) & ((output.nutrient=="folic acid") | (output.value_description=="percent of the population eating industrially produced vehicle"))]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,...,reg_A,updated,reg_B,reg_C,lower,upper,new_estimation_status,new_value_mean,new_value_025_percentile,new_value_975_percentile
123,180,Kenya,NaN,maize flour,percent of population eating fortified vehicle,folic acid,0.000000,0.0,0.000000,True,...,53.105833,False,36.67,0.563807,0.000000,0.000000,NaN,NaN,NaN,NaN
132,180,Kenya,NaN,wheat flour,percent of population eating fortified vehicle,folic acid,27.455666,0.0,53.159813,True,...,36.580000,False,NaN,NaN,17.074248,52.332418,multiplicative + rejection sampling,27.455666,0.0,53.159813


In [182]:
output[(output.location_name=="Kenya") & (output.vehicle.isin(['maize flour','wheat flour'])) & ((output.value_description=="percent of population eating industrially produced vehicle"))]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,u5_applicable,...,reg_A,updated,reg_B,reg_C,lower,upper,new_estimation_status,new_value_mean,new_value_025_percentile,new_value_975_percentile
122,180,Kenya,total population,maize flour,percent of population eating industrially prod...,na,36.670000,17.221861,56.118139,True,...,53.105833,False,36.67,0.563807,15.438217,57.901783,NaN,NaN,NaN,NaN
131,180,Kenya,total population,wheat flour,percent of population eating industrially prod...,na,36.709986,11.989033,62.573855,True,...,36.580000,False,NaN,NaN,15.227586,46.672414,na + rejection sampling,36.709986,11.989033,62.573855


In [64]:
break

SyntaxError: 'break' outside loop (<ipython-input-64-6aaf1f276005>, line 4)

In [ ]:
output[final_cols + ['u5_applicable','wra_applicable']].to_csv(all_inputted_data_path, index = False)
output[final_cols + ['u5_applicable','wra_applicable']].to_csv('/ihme/homes/beatrixh/repos/scratch/lsff_input_coverage_data.csv', index = False)

## generate data richness scores

In [65]:
ref2[['A_estimation_status','B_estimation_status','C_estimation_status']].drop_duplicates()

,A_estimation_status,B_estimation_status,C_estimation_status
0,regression,regression,regression
4,regression,na,regression
14,na,na,na
17,na,na,regression
28,na,na,multiplicative
29,na,multiplicative,multiplicative
40,na,na,lack of evidence
71,na,multiplicative,regression
73,na,multiplicative + rejection sampling,na + rejection sampling
87,na,na + rejection sampling,multiplicative + rejection sampling


In [66]:
ref2['score_from'] = ref2.A_estimation_status + '/' + ref2.B_estimation_status + '/' + ref2.C_estimation_status

In [67]:
score_map = {
    'regression/regression/regression': 0,
     'regression/na/regression': 1,
     'na/na/na': 5,
     'na/na/regression': 3,
     'na/na/multiplicative': 4,
     'na/multiplicative/multiplicative': 3,
     'na/na/lack of evidence': 4,
     'na/multiplicative/regression': 1,
     'na/multiplicative + rejection sampling/na + rejection sampling': 0,
     'na/na + rejection sampling/multiplicative + rejection sampling': 0,
     'regression/na/na': 5,
     'na/multiplicative/na': 4,
     'na/regression/multiplicative': 3,
     'na/regression/regression': 1,
     'na/na + rejection sampling/regression': 2,
     'na/na + rejection sampling/multiplicative': 2,
     'regression/regression/na': 2,
     'regression/regression + rejection sampling/regression + rejection sampling': 0,
     'na/regression/na': 4
    }

In [68]:
convert_4_pts = {
    0:0,
    1:1,
    2:2,
    3:2,
    4:3,
    5:4
}

In [69]:
convert_4_pts[score_map['na/na/na']]

4

In [70]:
ref2['score'] = ref2.score_from.map(score_map).map(convert_4_pts)

In [72]:
ref2[['location_name','vehicle','nutrient','score']]

,location_name,vehicle,nutrient,score
0,Ethiopia,maize flour,folic acid,0
1,Ethiopia,maize flour,iron,0
2,Ethiopia,maize flour,vitamin a,0
3,Ethiopia,maize flour,zinc,0
4,Ethiopia,oil,vitamin a,1
...,...,...,...,...
220,Sudan,oil,vitamin a,4
221,Sudan,wheat flour,folic acid,0
222,Sudan,wheat flour,iron,0
223,Sudan,wheat flour,vitamin a,0


In [83]:
location_names = ['Ethiopia',
 'India',
 'Nigeria',
 'Bangladesh',
 'Pakistan',
 'Uganda',
 'United Republic of Tanzania',
 'South Africa',
 'Burkina Faso',
 'Kenya',
 'Myanmar',
 'Nepal',
 'Vietnam',
 'Cameroon',
 "Côte d'Ivoire",
 'Democratic Republic of the Congo',
 'Indonesia',
 'Mozambique',
 'Angola',
 'China',
 'Egypt',
 'Ghana',
 'Madagascar',
 'Niger',
 'Sudan']

In [86]:
vehicles = ['wheat flour','oil','maize flour']
nutrients = ['folic acid','iron','zinc','vitamin a']

In [87]:
## load legal combos
import pickle
data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_vehicle_country_pairs.pickle', 'rb') as handle:
    vehicle_country_map = pickle.load(handle)

In [88]:
target_a = pd.DataFrame([(loc,vehicle,nutrient,'percent of population eating fortified vehicle') for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient','value_description']).sort_values(['location_name','vehicle','nutrient'])
target_a = target_a[(target_a.nutrient.isin(nutrients))]

target_b = pd.DataFrame([(loc,vehicle,'na',val) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for val in ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']],
            columns=['location_name','vehicle','nutrient','value_description'])

sortvars = ['location_name','vehicle','value_description','nutrient']
target = target_a.append(target_b)
target = target[(target.vehicle.isin(vehicles)) & (target.nutrient.isin(nutrients + ['na']))].sort_values(sortvars).set_index(sortvars)

In [110]:
frame = pd.DataFrame([(loc,vehicle,nutrient) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient']).sort_values(['location_name','vehicle','nutrient'])

In [111]:
frame.loc[(frame.location_name=="Vietnam"),'location_name'] = "Viet Nam"

In [112]:
frame = frame.merge(ref2[['location_name','vehicle','nutrient','score']], on = ['location_name','vehicle','nutrient'], how = 'left')

In [113]:
frame = frame[(frame.vehicle.isin(vehicles)) & (frame.nutrient.isin(nutrients))]

In [114]:
frame[frame.score.isna()]

,location_name,vehicle,nutrient,score


In [115]:
frame

,location_name,vehicle,nutrient,score
0,Angola,maize flour,folic acid,0.0
1,Angola,maize flour,iron,0.0
2,Angola,maize flour,vitamin a,0.0
5,Angola,maize flour,zinc,0.0
6,Angola,oil,vitamin a,0.0
...,...,...,...,...
375,Viet Nam,wheat flour,iron,2.0
376,Viet Nam,wheat flour,vitamin a,2.0
377,Viet Nam,wheat flour,vitamin a,2.0
380,Viet Nam,wheat flour,zinc,2.0


In [107]:
frame.to_csv('/ihme/homes/beatrixh/repos/scratch/scores.csv', index = False)

In [109]:
frame.score = frame.score.astype(int)

In [ ]:
ref2.A_estimation_status.unique()

In [ ]:
ref2.B_estimation_status.unique()

In [ ]:
ref2.C_estimation_status.unique()

In [ ]:
df[(df.value_mean.isna())]

In [ ]:
## rejection sampling

In [ ]:
df.columns

In [ ]:
df.loc[(df.location_name.isin(['Angola', 'China', 'Ghana', 'Niger', 'Egypt', 'Sudan', 'Madagascar']))][['location_name', 'sub_population', 'vehicle',
       'value_description', 'nutrient', 'value_mean', 'value_025_percentile',
       'value_975_percentile', 
       'estimation_status', 'source_citation', 'source_link', 'CI_source',]].to_csv('/ihme/homes/beatrixh/repos/scratch/tier5_3_31_2021.csv', index = False)

In [ ]:
# df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
#    (df.value_description=="percent of population eating indusrially produced vehicle") & 
#    (df.reg_A.notna()),'estimation_status'] = 'regression'

# df.loc[((df.estimation_status=="regression") | (df.value_mean.isna())) &
#    (df.value_description=="percent of population eating vehicle") & 
#    (df.reg_A.notna()),'value_mean'] = df.reg_A

In [ ]:
# impute A
ref = ref.merge(eats, on = ['location_name','vehicle'], how = 'outer')
ref.loc[(ref.A.isna()),'A'] = ref.reg_A

In [ ]:
df = ref.copy()

In [ ]:
# merge on B and filter
est = pd.concat([pd.read_csv(path) for path in paths_B])

est = est.groupby(['location_name','vehicle']).mean().reset_index()

df = df.merge(est, on = ['location_name','vehicle'], how = 'outer')

draws = [f'draw_{i}' for i in range(500)]

for draw in draws:
    df.loc[(df.A < df[draw]) | 
                 (df.C > df[draw]),draw] = np.nan

df['reg_B'] = df[draws].mean(axis=1)
df = df.drop(columns = draws)

# these are rows you still dont have estimates for
# df[(df.B.isna()) & (df.reg_B.isna())]

df.loc[(df.B.isna()) | (df.B_estimation_status=='regression'),'B'] = df.reg_B

In [ ]:
df[(df.A < df.B) | (df.B < df.C)]

In [ ]:
df[(df.B < df.C) & (df.C_estimation_status=="regression")]

In [ ]:
# merge on C and filter
est = pd.concat([pd.read_csv(path) for path in paths_C])

est = est.groupby(['location_name','vehicle']).mean().reset_index()

df = df.merge(est, on = ['location_name','vehicle'], how = 'outer')

draws = [f'draw_{i}' for i in range(500)]

for draw in draws:
    df.loc[(df.A < df[draw]) | 
                 (df.B < df[draw]),draw] = np.nan

df['reg_C'] = df[draws].mean(axis=1)
df = df.drop(columns = draws)

# these are rows you still dont have estimates for
# df[(df.B.isna()) & (df.reg_B.isna())]

df.loc[(df.C.isna()) | (df.C_estimation_status=='regression'),'C'] = df.reg_C

In [ ]:
df[(df.B < df.C) & (df.C_estimation_status=="regression")]

In [ ]:
df[(df.A < df.B) | (df.B < df.C)].shape

In [ ]:
ref[(ref.A < ref.B) | (ref.B < ref.C)].shape

In [ ]:
df[(df.A < df.B)]

In [ ]:
df[(df.B < df.C)]

In [ ]:
df[(df.A.isna() | df.B.isna() | df.C.isna())]

In [ ]:
df

In [ ]:
temp_a = df[['location_name','vehicle','nutrient','A']].rename(columns = {'A':'value_mean_est'}).drop_duplicates()
temp_a['value_description'] = "percent of the population eating vehicle"

temp_b = df[['location_name','vehicle','nutrient','B']].rename(columns = {'B':'value_mean_est'}).drop_duplicates()
temp_b['value_description'] = "percent of the population eating industrially produced vehicle"

temp_c = df[['location_name','vehicle','nutrient','C']].rename(columns = {'C':'value_mean_est'}).drop_duplicates()
temp_c['value_description'] = "percent of the population eating fortified vehicle"

In [ ]:
imputed = pd.concat([temp_a,temp_b,temp_c])

In [ ]:
imputed.head()

In [ ]:
original = pd.read_csv(w_metadata_path)

In [ ]:
test = original.merge(imputed, on = ['location_name','vehicle','value_description','nutrient'], how = 'outer')

In [ ]:
test.loc[((test.value_mean.isna()) | (test.estimation_status=="regression")) & (test.value_mean_est.notna())][['location_name','vehicle','value_description','nutrient','value_mean_est']]

In [ ]:
imputed.shape, original.shape

In [ ]:
original.location_name.nunique()

In [ ]:
imputed.location_name.nunique()

In [ ]:
imputed

In [ ]:
imputed['ac'] = imputed.groupby(['location_name','vehicle','value_description','nutrient']).transform('count').value_mean_est

In [ ]:
imputed[(imputed.ac > 1)]

In [ ]:
imputed[(imputed.ac > 1)].drop_duplicates()

In [ ]:
test[test.value_mean_est > 1]

In [ ]:
test.value_mean_est.min(), test.value_mean_est.max()